<a href="https://colab.research.google.com/github/alexrofail/Loss-Optimizers-Training-Loops/blob/main/training_mini_batches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn.functional as F
import gzip
import pickle
from torch import tensor
from torch import nn

In [2]:
def get_data():
    with gzip.open('mnist.pkl.gz') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))


In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
import pickle, gzip, math, torch, matplotlib as mpl

In [5]:
mpl.rcParams['image.cmap'] = 'gray'

In [6]:
n, m = x_train.shape
c = y_train.max()+1
nh = 50

In [7]:
class Model(nn.Module):
  def __init__(self, nin, nh, nout):
    super().__init__()
    self.layers = [nn.Linear(nin,nh), nn.ReLU(), nn.Linear(nh,nout)]

  def __call__(self, x):
    for l in self.layers: x = l(x)
    return x

In [8]:
model = Model(m, nh, 10)

In [9]:
pred = model(x_train)

In [11]:
#Now we need a loss function

In [14]:
#For loss funcs we need to first compute the softmax of our activations
def log_softmax(x):
  return (x.exp()/(x.exp().sum(-1, keepdim=True))).log() #NLL requires log softmax in Pytorch

In [15]:
sm_pred = log_softmax(pred)

In [16]:
#Cross Entropy Loss
#Sum of the actual * log prob(actual)
#But in this case, our actuals are 1-hot-encoded, it can be done as -log(p_sub_i) where sub_i is he index of desired target

In [22]:
#First three elements of dependent var
y_train[:3]

tensor([5, 0, 4])

In [20]:
sm_pred[[0,1,2], [5,0,4]]

tensor([-2.4577, -2.0858, -2.3811], grad_fn=<IndexBackward>)

In [21]:
y_train.shape

torch.Size([50000])

In [23]:
def nll(input, target): return -input[range(target.shape[0]), target].mean()

In [25]:
loss = nll(sm_pred, y_train)
loss

tensor(2.2899, grad_fn=<NegBackward>)

In [26]:
#We can refactor log_softmax(x)
def log_softmax(x):
  return x - x.exp().sum(-1, keepdim=True).log()